### MY UNDERSTANDING OF CONVNETS

Given an image (32x32x3 for example) we can set up a convolution of 5x5x3
Take the dot product between the convolution and chunks of the image


Conv layer comprises of a set of independent filters - feature maps
Unlike normal NN, each neuron is connected to only small chunk of input image 
and nerons have the same connection weights

<hr/>

<code>'batch_size'</code>          -> How many images at once do we want to pass 
                      -> (20 - 200) for smallish (~25,0000) dataset before any weights are updated

<code>'validation_split'</code>    -> Out of sample data


**Pooling**               -> Progressively reduce size of the layers to reduce computation
Max Pool              -> Take largest number in certain filter chunk

**Fully Connected Layer** -> At end of CNN, neurons have connections to all activatons in previous layer (Think NN)

**Dense layer**           -> Regular layer of neurons in NN with certain 'activation' (relu, sigmoid, tanh)
                      -> It's the fully connected layer usually found at the end of the architecture

**Dropout**               -> Randomly drops fraction of neurons
                      -> Imitates dead neurons
                      -> Used to counter Overfitting
<hr />
Sample Architecture/Workflow
 - Start with Input image (Car for example at 32x32 with Conv 2x2 stride 2)
 - Conv + Relu
 - Max Pooling -> Downsizes matrix to 8x8 for example
 - Conv + Relu
 - Max Pooling -> Downsizes matrix to 2x2 for example
 - Vectorize current Matrix
 - Add Fully Connected Layer
 - Apply Softmax to get final probabilities of classifications


In [1]:
#CONVNET FOLLOWING ARCHITECTURE SIMILAR TO THE ONE DESCRIBED ABOVE

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

#MAP X DATA FROM (0,255) TO (0,1)
X = X / 255.0

#2x64 ConvNet
model = Sequential()

#CONVNET + RELU
#@param: idk, convolution, input shape
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

#MAX POOL
model.add(MaxPooling2D(pool_size=(2,2)))

#CONVNET + RELU
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

#MAX POOL
model.add(MaxPooling2D(pool_size=(2,2)))

#FULLY CONNECTED LAYER
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss = "binary_crossentropy", 
              optimizer = "adam",
              metrics = ["accuracy"]
             )

model.fit(X, y, batch_size = 32, validation_split = 0.1, epochs = 3)

Using TensorFlow backend.


EOFError: Ran out of input

In [16]:
print(model.input_shape)
print(X.shape)
model.summary()

(None, 50, 50, 3)
(24946, 50, 50, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 64)        1792      
_________________________________________________________________
activation_5 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_6 (Activation)    (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 774

In [5]:
model.save('DogsCats.h5')
print(model.input_shape)

(None, 50, 50, 3)


In [20]:
import keras
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import coremltools


scale = 1/255.
coreml_model = coremltools.converters.keras.convert('DogsCats.h5',
                                               input_names=['image'], output_names=['output'], 
                                                image_input_names='image')
coreml_model.author = 'Sachin Katyal'
coreml_model.short_description = 'Dog vs Cat Recognition'
coreml_model.input_description['image'] = 'Takes as input an image of a Dog or Cat digit'
coreml_model.output_description['output'] = 'Prediction of Animal'
coreml_model.license = 'My ass'

coreml_model.save("DogsCats.mlmodel")

#coremltools.utils.save_spec(core_model, 'model.mlmodel')



0 : conv2d_1_input, <keras.engine.input_layer.InputLayer object at 0x1280903c8>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x1280900b8>
2 : activation_1, <keras.layers.core.Activation object at 0x128090358>
3 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x128090320>
4 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x138a568d0>
5 : activation_2, <keras.layers.core.Activation object at 0x138a6e4e0>
6 : max_pooling2d_2, <keras.layers.pooling.MaxPooling2D object at 0x138a6e8d0>
7 : flatten_1, <keras.layers.core.Flatten object at 0x128090518>
8 : dense_1, <keras.layers.core.Dense object at 0x135cc9828>
9 : activation_3, <keras.layers.core.Activation object at 0x138a6e898>
10 : dense_2, <keras.layers.core.Dense object at 0x135ce0748>
11 : activation_4, <keras.layers.core.Activation object at 0x135ce0cc0>
